In [ ]:
# Randomized statistical testing

In [ ]:
import numpy as np
import os
import subprocess
import sys
import random
import timeit

In [ ]:
qrels_file = './bioasq_dir/bioasq_test_qrels'
trec_eval_command = '../../eval/trec_eval'

run_dev_algo_A = './bioasq_dir/run_bioasq_best_lmart_test_leaves15_lr0.07_n750'
run_dev_algo_B = './bioasq_dir/run_bm25_bioasq_test_filtered'

In [ ]:
arr = list(np.random.randint(2, size=(10,)))

In [ ]:
def comp(x):
    return 1 - abs(x)

In [36]:
def meanAP(dict_X):
    return np.mean([dict_X[x] for x in  dict_X.keys()])

In [ ]:
def list_to_str(int_list):
    string = ""
    int_list = [str(x) for x in int_list] 
    return string.join(int_list)

In [ ]:
def get_run_avgs(trec_eval_command, qrel, qret):
    
    params = ['-q', '-m', 'map']
    toolkit_parameters = [
                            trec_eval_command,
                            *params,
                            qrel,
                            qret]

    print(toolkit_parameters)

    proc = subprocess.Popen(toolkit_parameters, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=False)
    (out, err) = proc.communicate()
#     print(out.decode("utf-8"))
    print('Run error: ', err)
    if err == None:
        pass
#         print('No errors')
    out_split = out.decode("utf-8").replace('\tall\t','').splitlines()[:-1]
    out_dict = {item.split()[1]:float(item.split()[2]) for item in out_split}
    return out_dict

In [ ]:
def get_paired_list(dict_A, dict_B):
    if not (set(dict_A.keys())  == set(dict_B.keys())):
        print('Queries sets are different!')
        return
    paired_list = []
    for k in dict_A.keys():
        paired_list.append([k, dict_A[k], dict_B[k]])
    return paired_list

In [ ]:
def permute_paired_list(paired_list, b_filter):    
    return [[p[0], p[b+1], p[comp(b)+1]] for p,b in zip(paired_list, b_filter)]

In [ ]:
dict_A = get_run_avgs(trec_eval_command, qrels_file, run_dev_algo_A)
dict_B = get_run_avgs(trec_eval_command, qrels_file, run_dev_algo_B)

In [ ]:
paired_list = get_paired_list(dict_A, dict_B)

In [37]:
map_A = meanAP(dict_A)
map_B = meanAP(dict_B)

In [34]:
start_time = timeit.default_timer()

max_iter = 10000

# initial_b_filter = [1] * 400
seen_filters = set()
for i in range(0, max_iter):
    b_filter = list(np.random.randint(2, size=(len(paired_list),)))

    while list_to_str(b_filter) in seen_filters:
        b_filter = list(np.random.randint(2, size=(len(paired_list),)))
        print('repeated')
   
    permute_paired_list(paired_list, b_filter)
    seen_filters.add(list_to_str(b_filter))
    
    
print(timeit.default_timer() - start_time)

6.908755868673325


10000